In [1]:
import io
import pandas as pd
from pandas import json_normalize

import numpy as np
import json
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False,nb_workers=2)

import gspread as gs
from datetime import datetime

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
import sys
import os

cur_directory=os.getcwd()
parent_directory=cur_directory[:cur_directory.rfind('/')]
sys.path.append(parent_directory)
 
# importing
import MyFunc
from MyFunc import *

from importlib import reload
reload(MyFunc)
from MyFunc import *

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [3]:
cashbac_key='Cashbac-GCP-Keys.json'
horego_key='Horego-GCP-Keys.json'

exclude_restorant=get_exclude_restorant(horego_key)

exclude_restorant_condition=''
for place_id in exclude_restorant.place_id.unique():
    exclude_restorant_condition=exclude_restorant_condition+' '+f"""or lower(place_id) = lower('{place_id}')"""

In [5]:
import google.auth
from google.oauth2 import service_account
from google.cloud import bigquery

credentials = service_account.Credentials.from_service_account_file('Cashbac-GCP-Keys.json')
project_id = 'cashbac-31433'

# Make clients.
cashbac_bqclient = bigquery.Client(
    credentials=credentials,
    project=project_id,
)

In [6]:
%%time
#######################################################################################

query_string=f""" 

SELECT * 
FROM 
(
    select * from cashbac-31433.cashbac_datalake_prod.outcrapper_google_results 
    union all 
    SELECT * FROM cashbac-31433.cashbac_datalake_prod.outcrapper_google_results_mall
)table1
where 
    {exclude_restorant_condition[3:]}
"""

query_result = list(cashbac_bqclient.query(query_string).result(timeout=None))
print(len(query_result))
if len(query_result)>0:
    data = pd.DataFrame(data=[list(x.values()) for x in query_result], columns=list(query_result[0].keys()))

del query_result

2681
CPU times: user 595 ms, sys: 90.5 ms, total: 686 ms
Wall time: 8.06 s


In [7]:
where_condition=''
for google_id in tqdm(data.google_id.unique()):
    where_condition=where_condition+' '+f"""or lower(google_id) like lower('%{google_id}%')"""

delete_query=f"""
    DELETE FROM `cashbac-31433.cashbac_datalake_prod.outcrapper_google_results`
    where{where_condition[3:]}
"""
query_job=cashbac_bqclient.query(delete_query).result(timeout=None)

delete_query=f"""
    DELETE FROM `cashbac-31433.cashbac_datalake_prod.outcrapper_google_results_mall`
    where{where_condition[3:]}
"""
query_job=cashbac_bqclient.query(delete_query).result(timeout=None)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1917/1917 [00:00<00:00, 127257.03it/s]


In [8]:
%%time

query_string=f""" 

SELECT column_name
FROM cashbac-31433.cashbac_datalake_prod.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'list_delete_in_prod'

"""

query_result = list(cashbac_bqclient.query(query_string).result(timeout=None))
df_temp = pd.DataFrame(data=[list(x.values()) for x in query_result], columns=list(query_result[0].keys()))

my_schema=[]
save_columns=[]
for column in df_temp.column_name :
    if column in data.columns :
        save_columns.append(column)
        my_schema.append(bigquery.SchemaField(column, "STRING"))

data[save_columns].drop_duplicates().to_csv('results/input_list_delete_in_prod.csv',sep=';',index=False)

CPU times: user 37.7 ms, sys: 21.3 ms, total: 59 ms
Wall time: 853 ms


In [9]:
%%time
cashbac_key='Cashbac-GCP-Keys.json'
horego_key='Horego-GCP-Keys.json'

#############################################################################################

src_path_file='results/input_list_delete_in_prod.csv'
target_path_file='scrapping_results/input_list_delete_in_prod.csv'
bucket_name='cashbac_datalake'
upload_file_to_GCS(cashbac_key,bucket_name,src_path_file,target_path_file)

table_schema = "cashbac_datalake_prod"
table_name = "list_delete_in_prod"
gcs_path_uri = f"gs://{bucket_name}/scrapping_results/input_list_delete_in_prod.csv"
create_table_from_gcs(cashbac_key,gcs_path_uri,table_schema,table_name,my_schema,'append')

Uploading results/input_list_delete_in_prod.csv
Path in GCS: cashbac_datalake/scrapping_results/input_list_delete_in_prod.csv
table on : cashbac_datalake_dev.list_delete_in_prod
CPU times: user 320 ms, sys: 20.6 ms, total: 341 ms
Wall time: 3.47 s


In [44]:
%%time

query_string=f""" 

SELECT column_name
FROM cashbac-31433.cashbac_datalake_prod.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'list_delete_in_prod'

"""

query_result = list(cashbac_bqclient.query(query_string).result(timeout=None))
df_temp = pd.DataFrame(data=[list(x.values()) for x in query_result], columns=list(query_result[0].keys()))

my_schema=[]
save_columns=[]
for column in df_temp.column_name :
    if column in data.columns :
        save_columns.append(column)
        my_schema.append(bigquery.SchemaField(column, "STRING"))

data[save_columns].drop_duplicates().to_csv('results/input_list_delete_in_prod.csv',sep=';',index=False)

CPU times: user 26.8 ms, sys: 36.9 ms, total: 63.7 ms
Wall time: 1.14 s


In [46]:
%%time
cashbac_key='Cashbac-GCP-Keys.json'
horego_key='Horego-GCP-Keys.json'

#############################################################################################

src_path_file='results/input_list_delete_in_prod.csv'
target_path_file='scrapping_results/input_list_delete_in_prod.csv'
bucket_name='cashbac_datalake'
upload_file_to_GCS(cashbac_key,bucket_name,src_path_file,target_path_file)

table_schema = "cashbac_datalake_prod"
table_name = "list_delete_in_prod"
gcs_path_uri = f"gs://{bucket_name}/scrapping_results/input_list_delete_in_prod.csv"
create_table_from_gcs(cashbac_key,gcs_path_uri,table_schema,table_name,my_schema,'append')

Uploading results/input_list_delete_in_prod.csv
Path in GCS: cashbac_datalake/scrapping_results/input_list_delete_in_prod.csv
table on : cashbac_datalake_prod.list_delete_in_prod
CPU times: user 336 ms, sys: 35 ms, total: 371 ms
Wall time: 3.6 s


##### DELETE UNWANTED NAME

In [8]:
try:
    cursor.close()
    conn_details.close()
except:
    pass
    
conn_details=db_connect_cashbac_nopool('uat-cashbac-db.ciwxizsaix0m.ap-southeast-1.rds.amazonaws.com','cashbacdb','data_ds','data123')
cursor = conn_details.cursor()

for name in tqdm(name_exclude.KEYWORD):

    delete_query=f"""
        DELETE FROM cbmerchant.outlet
        WHERE lower(outlet_name) like lower('%{name}%')
    """

    cursor.execute(delete_query)
    conn_details.commit()
    
cursor.close()
conn_details.close()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:13<00:00,  2.79it/s]


In [13]:
# where_condition=''
# for name in tqdm(name_exclude.KEYWORD.unique()):
#     where_condition=where_condition+' '+f"""or lower(name) like lower('%{name}%')"""

# delete_query=f"""
#     DELETE FROM `cashbac-31433.cashbac_datalake_dev.outcrapper_google_results`
#     where{where_condition[3:]}
# """
# query_job=cashbac_bqclient.query(delete_query).result(timeout=None)

# delete_query=f"""
#     DELETE FROM `horego-big-query.horego_datalake_dev.outcrapper_google_results`
#     where{where_condition[3:]}
# """
# query_job=horego_bqclient.query(delete_query).result(timeout=None)

In [12]:
name_exclude.KEYWORD[49]

"de'cafe (car wash- fl detailing- cafe - vape shop)"

In [14]:
# for name in tqdm(name_exclude.KEYWORD):
    
#     delete_query=f"""
#         DELETE FROM `cashbac-31433.cashbac_datalake_dev.outcrapper_google_results`
#         WHERE lower(name) like lower('%{name}%')
#     """
#     query_job=cashbac_bqclient.query(delete_query).result(timeout=None)
    
#     delete_query=f"""
#         DELETE FROM `horego-big-query.horego_datalake_dev.outcrapper_google_results`
#         WHERE lower(name) like lower('%{name}%')
#     """
#     query_job=horego_bqclient.query(delete_query).result(timeout=None)